In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

from tqdm import tqdm
import re

#import for another version using NLTK
import datetime
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer# from sklearn.xvm import SVC
from nltk.tokenize import TweetTokenizer



In [ ]:
!pip install tensorflow_text

In [ ]:
#loading data 
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
#sampling over data train
train_arrays, test_arrays, train_labels, test_labels = train_test_split(X_train,y_train,test_size=0.05)

## SVM

In [ ]:
# Normalize(Remove stop words, stemmatize) tweets

stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

def normalizer(tweet):
    only_letters = re.sub("[^a-zA-Z]", " ",tweet) 
    tokens = word_tokenize(only_letters)[2:]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas

train['normalize_text'] = train['text'].apply(normalizer)

#train.head()
print (train.target.value_counts())

# svc  = SVC()
# svc.fit(X_train, Y_train)
# predictions = svc.predict(X_test)

# print(classification_report(y_test, predictions))

In [ ]:
#Use bi-gram/ tri-gram
from nltk import ngrams
def ngrams(input_list):
    #onegrams = input_list
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams+trigrams

normalize_text_aggr_arr = []
for row in train['normalize_text']:
    aggr_str = " ".join(row)
    normalize_text_aggr_arr.append(aggr_str) 

#ngram_normalize_text_aggr_arr = []
#for row in train['ngram_normalize_text']:
#    aggr_str = " ".join(row)
#    ngram_normalize_text_aggr_arr.append(aggr_str) 

train['normalize_text_aggr'] = normalize_text_aggr_arr
#train['ngram_normalize_text_aggr'] = normalize_text_aggr_arr
#train[["text","normalized_text","normalize_text_aggr", "ngram_normalize_text_aggr"]]
# train['ngram_normalize_text'] = train['normalize_text'].apply(ngrams)
train.head()

In [ ]:
#Use word tokenizer 
train['word_tokenized_text'] = train['text'].apply(word_tokenize)

#Use tweet tokenizer
tt = TweetTokenizer()
train['tweet_tokenized_text'] = train['text'].apply(tt.tokenize)

train.head()

In [ ]:
import collections
def count_words(input):
    cnt = collections.Counter()
    for row in input:
        for word in row:
            cnt[word] += 1
    return cnt

#Most frequent words for disaster tweets
dict1 = train[train.target == 1][['ngram_normalize_text']].apply(count_words).ngram_normalize_text
print (dict1.most_common(30))


In [ ]:
#Most frequent words for disaster tweets
dict1 = train[train.target == 1][['normalize_text']].apply(count_words).normalize_text
print (dict1.most_common(30))

In [ ]:
#Most frequent words for NON-disaster tweets
dict1 = train[train.target == 0][['ngram_normalize_text']].apply(count_words).ngram_normalize_text
print (dict1.most_common(30))

In [ ]:
#Most frequent words for disaster tweets
dict1 = train[train.target == 1][['word_tokenized_text']].apply(count_words).word_tokenized_text
print (dict1.most_common(30))

In [ ]:
#Most frequent words for disaster tweets
dict1 = train[train.target == 1][['tweet_tokenized_text']].apply(count_words).tweet_tokenized_text
print (dict1.most_common(30))

In [ ]:

# Vectorization - normalized
vectorized_data = count_vectorizer.fit_transform(train.normalize_text_aggr)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))
targets = train['target']
print (indexed_data)

In [ ]:
# #Vectorization
# vectorized_data = count_vectorizer.fit_transform(train.text)
# indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))
# targets = train['target']
#print (indexed_data)

In [ ]:
#split train and validation data
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.4, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [ ]:
#fitting svm classfier
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
svm_clf =svm.SVC(kernel = 'linear')

svm_clf_output = svm_clf.fit(data_train, targets_train)
svm_clf.score(data_train, targets_train)

In [ ]:
svm_clf.score(data_test, targets_test)

In [ ]:
#Making submission
test_pred = svm_clf.predict(X_test)
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)